In [26]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [27]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
from keras.datasets import mnist
import pandas as pd

class XOR(nn.Module):

  def __init__(self, config):
    super(XOR, self).__init__()

    # 입력층 노드 수
    self.inode = config["input_node"]
    # 은닉층 데이터 크기
    self.hnode = config["hidden_node"]
    # 출력층 노드 수: 분류해야 하는 레이블 수
    self.onode = config["output_node"]

    # 활성화 함수로 Sigmoid 사용
    self.activation = nn.Sigmoid()

    # 신경망 설계
    self.linear1 = nn.Linear(self.inode, self.hnode, bias=True)
    self.linear2 = nn.Linear(self.hnode, self.onode, bias=True)  

  def forward(self, input_features):

    # 이곳을 채우세요.
    output1=self.linear1(input_features)
    hypothesis1=self.activation(output1)

    output2=self.linear2(hypothesis1)
    hypothesis2=self.activation(output2)

    return hypothesis2

In [28]:
# 데이터 읽기 함수
def load_dataset(file):

  dataset = pd.read_csv(file)
  # dataset= np.loadtxt(file)
  data=dataset.values

  x=data[:,0:13]
  print("INPUT_FEATURES=",x)

  y=np.reshape(data[:,13],(2716,1))
  print("LABELS=",y)

  number_of_train = int(len(data)*0.8)

  train_x = x[:number_of_train]
  train_y = y[:number_of_train]
  test_x = x[number_of_train:]
  test_y = y[number_of_train:]

  train_X = torch.tensor(train_x, dtype=torch.float)
  train_y = torch.tensor(train_y, dtype=torch.float)
  test_X = torch.tensor(test_x, dtype=torch.float)
  test_y = torch.tensor(test_y, dtype=torch.float)

  return(train_X, train_y), (test_X, test_y)

In [29]:
# 모델 평가 결과 계산을 위해 텐서를 리스트로 변환하는 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

# 평가 수행 함수
def do_test(model, test_dataloader):

  # 평가 모드 셋팅
  model.eval()

  # Batch 별로 예측값과 정답을 저장할 리스트 초기화
  predicts, golds = [], []
  
  with torch.no_grad():

    for step, batch in enumerate(test_dataloader):
  
      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      input_features, labels = batch
      # 이곳을 채우세요.
      hypothesis=model(input_features)
      logits=(hypothesis>0.5).float()

      # # ont-hot 표현으로 변경
      # logits = torch.argmax(hypothesis,-1)

      x = tensor2list(logits)
      y = tensor2list(labels)

      # 예측값과 정답을 리스트에 추가
      predicts.extend(x)
      golds.extend(y)
    
    print("PRED=",predicts)
    print("GOLD=",golds)
    print("Accuracy= {0:f}\n".format(accuracy_score(golds, predicts)))

# 모델 평가 함수
def test(config):

  # 이곳을 채우세요.
  model=XOR(config).cuda()

  # 저장된 모델 가중치 로드
  model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"])))

  # 데이터 load
  (_, _), (features, labels) = load_dataset(config["input_data"])
  # (features, labels) = load_dataset(config["input_data"])
  
  test_features = TensorDataset(features, labels)
  test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])
  
  do_test(model, test_dataloader)

In [30]:
# 모델 학습 함수
def train(config):

  # 이곳을 채우세요.
  model=XOR(config).cuda()

  (input_features,labels), (_,_)=load_dataset(config["input_data"])
  # (input_features,labels)=load_dataset(config["input_data"])

  # TensorDataset/DataLoader를 통해 배치(batch) 단위로 데이터를 나누고 셔플(shuffle)
  train_features = TensorDataset(input_features, labels)
  train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

  # 이곳을 채우세요.
  loss_func=nn.BCELoss()
  # optimizer=torch.optim.SGD(model.parameters(),lr=config["learn_rate"])
  # loss_func=nn.CrossEntropyLoss()
  optimizer=torch.optim.Adam(model.parameters(),lr=config["learn_rate"])

  for epoch in range(config["epoch"]+1):

    # 학습 모드 셋팅
    model.train()
    
    # epoch 마다 평균 비용을 저장하기 위한 리스트
    costs = []

    for (step, batch) in enumerate(train_dataloader):

      # batch = (input_features[step], labels[step])*batch_size
      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      # 각 feature 저장
      input_features, labels = batch

      # 역전파 변화도 초기화
      # .backward() 호출 시, 변화도 버퍼에 데이터가 계속 누적한 것을 초기화
      optimizer.zero_grad()

      # 이곳을 채우세요.
      hypothesis=model(input_features)
      cost=loss_func(hypothesis,labels)

      # 역전파 수행
      cost.backward()
      optimizer.step()
   
      # 현재 batch의 스텝 별 loss 저장
      costs.append(cost.data.item())
    
    # 에폭마다 평균 비용 출력하고 모델을 저장
    if epoch%100==0:
      print("Average Loss= {0:f}".format(np.mean(costs)))
      torch.save(model.state_dict(), os.path.join(config["output_dir"], "epoch_{0:d}.pt".format(epoch)))
      do_test(model, train_dataloader)

In [32]:
if(__name__=="__main__"):

    root_dir = "/gdrive/MyDrive/Colab/SleepHelper"
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    input_data="/gdrive/MyDrive/Colab/SleepHelper/dataSample.csv"

    config = {"mode": "train",
              "model_name":"epoch_{0:d}.pt".format(1000),
              "output_dir":output_dir,
              "input_data":input_data,
              "input_node":13,
              "hidden_node":6,
              "output_node":1,
              "learn_rate":0.1,
              "batch_size":50,
              "epoch":1000,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

INPUT_FEATURES= [[390.         270.          69.23076923 ...   0.           0.
    1.        ]
 [780.         165.          21.15384615 ...   2.           0.
    2.        ]
 [600.         395.          65.83333333 ...   2.           0.
    1.        ]
 ...
 [410.         370.          90.24390244 ...   0.           0.
    1.        ]
 [680.         355.          52.20588235 ...   0.           0.
    3.        ]
 [383.         281.          73.36814621 ...   0.           0.
    3.        ]]
LABELS= [[2.]
 [1.]
 [2.]
 ...
 [2.]
 [3.]
 [3.]]
Average Loss= -50.292209
PRED= [[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1